# Práctica 10: 

Elaboró: Carlos Alejandro Jarero Gonzalez <al255813@alumnos.uacj.mx>

Matrícula: 255813

El presente Notebook fue relizado en equipo local con Kernel Python 3.11.8 en VS Code.

## Objetivos

Analizar la distribución de las propinas (tip) con histogramas interactivos.

## Instrucciones

1. Cargar el dataset tips desde seaborn. 
```python
sns.load_dataset("tips")
```

In [363]:
import seaborn as sns
import plotly.express as px
from scipy.stats import shapiro, ks_2samp, kruskal
import statsmodels.formula.api as sm
import numpy as np

df = sns.load_dataset("tips")

df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [364]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [365]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


2. Usar plotly para graficar un histograma de las propinas (tip).

In [366]:
px.histogram(df, x="tip", histfunc="count"
             , title="Tips distribution")


3. Personalizar el número de bins y agregar colores según el día de la semana (day).

In [367]:
px.histogram(df
             , x="tip"
             , color="day"
             , color_discrete_sequence=px.colors.qualitative.Antique
             , histfunc="count"
             , nbins=10
             , title="Tips distribution")


# Extra

Revisamos normalidad de propinas

In [368]:
_, pvalue = shapiro(df["tip"])

print(f"p-value: {pvalue:.4f} => Because p<0.05, we reject the null hypothesis, and tip is not normally distributed")

p-value: 0.0000 => Because p<0.05, we reject the null hypothesis, and tip is not normally distributed


In [369]:
for _ in df["day"].unique():
    df_day = df[df["day"] == _]
    s, pvalue = shapiro(df_day["tip"])
    print(f"p-value of {_}: {pvalue:.4f}")

p-value of Sun: 0.0263
p-value of Sat: 0.0000
p-value of Thur: 0.0000
p-value of Fri: 0.8497


Por grupo solo encotnramos que el viernes se encuentra normalmente distribuido. Verifiquemos si los residuos siguen la normalidad.

In [370]:
model = smf.ols('tip ~ C(day)', data=df).fit()

res_day = df[["day"]]

res_day["resid"] = model.resid 

for e in res_day["day"].unique():
    resid_day = res_day.loc[res_day["day"] == e, "resid"]
    s, pvalue = shapiro(resid_day)
    print(f"Residuals p-value of {e}: {pvalue:.4f}")


Residuals p-value of Sun: 0.0263
Residuals p-value of Sat: 0.0000
Residuals p-value of Thur: 0.0000
Residuals p-value of Fri: 0.8497


Por residuos igual solo viernes se encuentra normalmente distribuido. Esto nos imposibilita hacer ANOVA. Si queremos ver si estádisticamente algún día dan más propina que otro podemos hacer un Kruskall-Wallis ya que no tenemos la distribución normal en tips.

Verificamos que no haya grupos menores a 5.

In [371]:
for _ in df["day"].unique():
    print(f"Grupo: {_} n = {df[df['day'] == _ ]['tip'].count()}")

Grupo: Sun n = 76
Grupo: Sat n = 87
Grupo: Thur n = 62
Grupo: Fri n = 19


Sin grupos menores a cinco. Verificamos que las distribuciones sean similares entre los grupos.

In [372]:
groups = res_day["day"].unique()

for i in range(len(groups)):
    for j in range(i+1,len(groups)):
        g1 = df[df["day"] == groups[i]]["tip"]
        g2 = df[df["day"] == groups[j]]["tip"]
        _, pv = ks_2samp(g1,g2)
        print(f"Group {groups[i]} vs {groups[j]} p={pv:.4f}")



Group Sun vs Sat p=0.0218
Group Sun vs Thur p=0.0136
Group Sun vs Fri p=0.3323
Group Sat vs Thur p=0.4365
Group Sat vs Fri p=0.9880
Group Thur vs Fri p=0.7140


Obvervamos que hay diferencias significativas entre Domingo y sabado y Domingo y jueves. Veamos la gradica de cajas.

In [373]:
px.box(df, y="tip", x="day")

Vemos outliners que pueden estar afectando niestros calculos en sabado y jueves. Solo como práctica haremos Kruskall-Wallis ya que sus supuestos son menos estrictos. 

In [374]:
g_sun = df[df["day"] == "Sun"]["tip"]
g_sat = df[df["day"] == "Sat"]["tip"]
g_thur = df[df["day"] == "Thur"]["tip"]
g_fri = df[df["day"] == "Fri"]["tip"]


k_stat, p_val = kruskal(g_sun,g_sat,g_thur,g_fri)

print(f"Kruskal-Wallis:  {k_stat}")
print(f"p: {p_val}")
print(f"median Sun (IQR): {np.median(g_sun):.4f} ({np.percentile(g_sun,25):.4f} - {np.percentile(g_sun, 75):.4f})")
print(f"median Sat (IQR): {np.median(g_sat):.4f} ({np.percentile(g_sat,25):.4f} - {np.percentile(g_sat, 75):.4f})")
print(f"median Fri (IQR): {np.median(g_fri):.4f} ({np.percentile(g_fri,25):.4f} - {np.percentile(g_fri, 75):.4f})")
print(f"median Thur (IQR): {np.median(g_thur):.4f} ({np.percentile(g_thur,25):.4f} - {np.percentile(g_thur, 75):.4f})")

Kruskal-Wallis:  8.565587588927054
p: 0.035660560194476144
median Sun (IQR): 3.1500 (2.0375 - 4.0000)
median Sat (IQR): 2.7500 (2.0000 - 3.3700)
median Fri (IQR): 3.0000 (1.9600 - 3.3650)
median Thur (IQR): 2.3050 (2.0000 - 3.3625)


Tomando en cuenta la falta de supuestos, podemos ver que los domingos son los mejroes dias para recibir propinas mientras que el jueves es el dia en que menos propinas reciben.